In [14]:
:dep darn
:dep ndarray
:dep ndarray-csv
:dep polars = { version = "x", features = ["csv","describe","fmt","lazy","ndarray","dtype-categorical","to_dummies"]}
:dep plotters = {version = "0.3.5", features = ["evcxr", "line_series"]}
:dep linfa
:dep linfa-svm
:sccache 1
    
extern crate csv;

use std::io::prelude::*;
use std::fs::*;
use std::path::Path
use ndarray::prelude::*;
use ndarray_csv::Array2Reader;
use ndarray::*;

use polars::prelude::*;
use std::fs::File;

use linfa::*;
use linfa::prelude::*;
use linfa_svm::{error::Result, Svm};

use linfa::traits::Transformer;

extern crate plotters;
use plotters::prelude::*;
use plotters::series::*;

sccache: true. Warning: dynamic linking disabled, use :cache instead to preserve dynamic linking


In [25]:
// Encontrar retas de hiperplanos, em decisoes de margem maxima
// 1/2 |w|2 + c SUM(ai)
// nao linear (kernel trick) -> linear

let df = CsvReader::from_path("./db/risco_credit.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

let df= df.clone().lazy();



let x = df.clone().select([col("*").exclude(["risco"])])
    .with_columns([
        col("historia").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("divida").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("garantias").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("renda").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical()     
    ])
    .drop_nulls(None).collect();
println!("DataFrame {:?}",x);


let y = df.clone()
    .select([col("risco")])
    .drop_nulls(None)
    .collect();

let y = y?.column("risco")?.clone();


{

let ndarray_x = x?.clone().to_ndarray::<Float32Type>(IndexOrder::C).unwrap();

let ndarray_y:Vec<String> =   y.str()?.
                                    into_iter().
                                    filter_map(|opt_str| opt_str.map(|s| s.to_string()))
                                    .collect();
    
let ds = Dataset::new(ndarray_x, ndarray_y.into());
    
    {
        let (train, test) = ds.split_with_ratio(1.);
        
    
    let params = Svm::<_, Pr>::params()
        //.pos_neg_weights(5000., 500.)
        .gaussian_kernel(30.0);

        let model = train
        .one_vs_all()?
        .into_iter()
        .map(|(l, x)| (l, params.fit(&x).unwrap()))
        .collect::<MultiClassModel<_, _>>();

                        

        
        //historia boa 0, divida alta 0. garantias nenhuma 1, renda > 35 2
        //historia ruim 2, divida alta 0, garantias adequada 0, remda < 15 0
        let test_values = array![[0.0,0.0,1.0,2.0],[2.0,0.0,0.0,0.0]];
        
        let predictions = model.predict(&test_values);
        



        
    }
}

DataFrame Ok(shape: (14, 4)
┌──────────┬────────┬───────────┬───────┐
│ historia ┆ divida ┆ garantias ┆ renda │
│ ---      ┆ ---    ┆ ---       ┆ ---   │
│ u32      ┆ u32    ┆ u32       ┆ u32   │
╞══════════╪════════╪═══════════╪═══════╡
│ 0        ┆ 0      ┆ 0         ┆ 0     │
│ 1        ┆ 0      ┆ 0         ┆ 1     │
│ 1        ┆ 1      ┆ 0         ┆ 1     │
│ 1        ┆ 1      ┆ 0         ┆ 2     │
│ 1        ┆ 1      ┆ 0         ┆ 2     │
│ …        ┆ …      ┆ …         ┆ …     │
│ 2        ┆ 0      ┆ 1         ┆ 2     │
│ 2        ┆ 0      ┆ 0         ┆ 0     │
│ 2        ┆ 0      ┆ 0         ┆ 1     │
│ 2        ┆ 0      ┆ 0         ┆ 2     │
│ 0        ┆ 0      ┆ 0         ┆ 1     │
└──────────┴────────┴───────────┴───────┘)


()

In [ ]:
let df = CsvReader::from_path("./db/census.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

let df= df.clone().lazy();



let x_census = df.clone().select([col("*").exclude(["income"])])
    .with_columns([
        col("workclass").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("final-weight").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("education").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("marital-status").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("occupation").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("relationship").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("race").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("sex").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("hour-per-week").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("capital-loss").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("capital-gain").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("education-num").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),


        col("native-country").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),        
    ])
    .drop_nulls(None).collect();
        println!("train {:?}",x_census);


let y_census = df.clone()
    .select([col("income")])
    .drop_nulls(None)
    .collect();

let y_census = y_census?.column("income")?.clone();


// let dummies_x = x_census?.clone().to_dummies(None, false).unwrap();

{

let ndarray_x = x_census?.clone().to_ndarray::<Float32Type>(IndexOrder::C).unwrap();

let ndarray_y:Vec<String> =   y_census.str()?.
                                    into_iter().
                                    filter_map(|opt_str| opt_str.map(|s| s.to_string()))
                                    .collect();
    
let ds = Dataset::new(ndarray_x, ndarray_y.into());
    
    {
        let (train, test) = ds.split_with_ratio(0.8);

        let params = Svm::<_, Pr>::params()
        //.pos_neg_weights(5000., 500.)
        .gaussian_kernel(30.0);

        let model = train
        .one_vs_all()?
        .into_iter()
        .map(|(l, x)| (l, params.fit(&x).unwrap()))
        .collect::<MultiClassModel<_, _>>();

                        
        let predictions = model.predict(&train);
        
        let cm = predictions.confusion_matrix(&test).unwrap();

        println!("Confusion Matrix ");
        println!("{:?}", predictions);
        println!("{:?}", cm);
        println!("{:?}", cm.accuracy());


        
    }
}